<a href="https://colab.research.google.com/github/chairiq/FoodCNNs/blob/master/source/YOLOv4_Training_on_CT_Scan_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This following pipline was developed based on the instructions and code of the two following repositories.
*   [AlexeyAB](https://github.com/AlexeyAB/darknet)
*   [theAIGuysCode](https://github.com/theAIGuysCode)



# Step 1: Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Cloning and Building Darknet



In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i '0,/assert(x < m.w && y < m.h && c < m.c)/s//\/\/assert(x \< m.w \&\& y \< m.h \&\& c \< m.c)/' src/image.c

# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# Step 3: Download pre-trained YOLOv4 weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

# Step 4: Define Helper Functions


In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/drive/MyDrive/ /mydrive 
!ls /mydrive

# Step 5) Train Your Own YOLOv4 Custom Object Detector!

In order to create a custom YOLOv4 detector we will need the following:

*   Labeled Custom Dataset
*   Custom .cfg file
*   ctscans.data and ctscans.names files
*   train.txt file (test.txt is optional here as well)

## Step 5.1: Moving Your Custom Datasets Into Your Cloud VM


In [ ]:
! cp -R /mydrive/yolov4/ctscans-train /content/darknet/data/ctscans-train
! cp -R /mydrive/yolov4/ctscans-test /content/darknet/data/ctscans-test

## Step 5.2: Copy Files for Training
Create a new file within a code or text editor called **ctscans.names** where you will have one class name per line.

1. Yes

You will also create a **ctscans.data** file and fill it in like this (change your number of classes accordingly, as well as your backup location)


1. class = 1
2. train = data/ctscans-train.txt
3. valid = data/ctscans-test.txt
4. names = data/ctscans.names
5. backup = /mydrive/yolo4/backup



In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov4/support-files/ctscans.names ./data
!cp /mydrive/yolov4/support-files/ctscans.data  ./data
!cp /mydrive/yolov4/support-files/yolov4-ctscans.cfg cfg/yolov4-ctscans.cfg

## iii) Generating train.txt and test.txt



In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolov4/support-files/generate_train.py ./
!cp /mydrive/yolov4/support-files/generate_test.py ./

In [ ]:
# Run both scripts to genereate the twio aformentioned txt files
!python generate_train.py
!python generate_test.py

# Step 6: Download pre-trained weights for the convolutional layers.


In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Step 7: Train Your Custom Object Detector!


In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/ctscans.data cfg/yolov4-ctscans.cfg yolov4.conv.137 -dont_show -ext_output -map


In [ ]:
# show chart.png of how custom object detector did with training and save to to the backup folder
imShow('chart.png')
!mv chart.png /mydrive/yolov4/backup/

# Step 9: Run Your Custom Object Detector!!!


In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-ctscans.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-ctscans.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/ctscans.data cfg/yolov4-ctscans.cfg /mydrive/yolov4/backup/yolov4-ctscans_last.weights /my_image.jpg -thresh 0.6
imShow('predictions.jpg')

Cite this work as:   